In [1]:
import numpy as np
import pandas as pd

In [2]:
projects_data = pd.read_excel("final_projects_2020_bmi.xlsx",engine='openpyxl')
projects_data.shape

(86888, 298)

In [3]:
gc_data = pd.read_excel("Global (World) Cities List - Beaverstock et al.xlsx")
gc_data.shape

(124, 8)

In [4]:
list(gc_data.columns)

['#',
 'GlobalCity',
 'Country',
 'Category',
 'Sub-category',
 'Description',
 'GC1',
 'GC2']

In [5]:
# Rectify city names in Global (World) Cities List - Beaverstock et al.xlsx to match

In [6]:
city_dict ={'AbuDhabi':'Abu Dhabi',
'Bogotá':'Bogota',
'CapeTown':'Cape Town',
'Düsseldorf':'Dusseldorf',
'HoChiMinhCity':'Ho Chi Minh City',
'HongKong':'Hong Kong',
'KansasCity':'Kansas City',
'KualaLumpur':'Kuala Lumpur',
'LosAngeles':'Los Angeles',
'MexicoCity':'Mexico City',
'NewDelhi':'New Delhi',
'NewYork':'New York',
'RiodeJaneiro':'Rio De Janeiro',
'SanFrancisco':'San Francisco',
'SãoPaulo':'Sao Paulo',
'St.Petersburg':'St Petersburg',
'TelAviv':'Tel Aviv',
'Zürich':'Zurich',
'BuenosAires':'Buenos Aires',
'TheHague':'The Hague',
'Brasília':'Brasilia',
}

In [7]:
for c in city_dict.items():
    indexes = gc_data.loc[gc_data.GlobalCity == c[0]].index
    gc_data.at[indexes,'GlobalCity'] =c[1]
   

In [8]:
list( set(gc_data['GlobalCity']) - set(projects_data['dcity']) )

[]

In [ ]:
# Many city names in project are spelled incorrectly refer to the projects_city_name_diff document
# Correcting the city names

In [9]:
gc_data.dropna(subset=['GlobalCity'])

,#,GlobalCity,Country,Category,Sub-category,Description,GC1,GC2
0,1.0,London,UK,Alpha,Alpha1,Full world city,1,1.0
1,2.0,New York,USA,Alpha,Alpha1,Full world city,1,1.0
2,3.0,Paris,France,Alpha,Alpha1,Full world city,1,1.0
3,4.0,Tokyo,Japan,Alpha,Alpha1,Full world city,1,1.0
4,5.0,Chicago,USA,Alpha,Alpha2,Full world city,1,1.0
...,...,...,...,...,...,...,...,...
118,119.0,Tehran,Iran,Delta,Delta3,Minimal evidence of world city formation,0,4.0
119,120.0,Tijuana,Mexico,Delta,Delta3,Minimal evidence of world city formation,0,4.0
120,121.0,Turin,Italy,Delta,Delta3,Minimal evidence of world city formation,0,4.0
121,122.0,Utrecht,Netherlands,Delta,Delta3,Minimal evidence of world city formation,0,4.0


Get list of diff city names. Most city names are misspelt so checkng for first 2 letter and manually removing any incorrect entries

In [12]:
gc_data.GlobalCity = gc_data.GlobalCity.astype(str)

In [10]:
projects_data['icity'] = projects_data['icity'].str.lower()
projects_data['icity']= projects_data['icity'].str.title() 
projects_data['dcity'] = projects_data['dcity'].str.lower()
projects_data['dcity']= projects_data['dcity'].str.title() 

# Run these only to generate new file with corrected city name. To reuse existing file skip this and goto to Main step 

In [18]:
i=[]
dcity=[]
for index,c in gc_data.iterrows():
    b = projects_data[(projects_data['icoun'] == c.Country)  & ((projects_data['icity'] != c['GlobalCity']) & (projects_data['icity'].str[:2] == c['GlobalCity'][:2]))][['icity','icoun']]
    b['GlobalCity'] = c['GlobalCity']
    i.append(b) 

for index,c in gc_data.iterrows():
    d = projects_data[(projects_data['dcoun'] == c.Country)  & ((projects_data['dcity'] != c['GlobalCity']) & (projects_data['dcity'].str[:2] == c['GlobalCity'][:2]))][['dcity','dcoun']]
    d['GlobalCity'] = c['GlobalCity']
    dcity.append(d)     


i_data = pd.concat(i)
d_data = pd.concat(dcity)
d_data.columns =['icity','icoun','GlobalCity']
df_merge = pd.concat([i_data, d_data], ignore_index=True)
df_merge

,icity,icoun,GlobalCity
0,Paris La Défense,France,Paris
1,Paris 15Eme,France,Paris
2,Paris La Defense,France,Paris
3,Paris 15Eme,France,Paris
4,Paris La Défense,France,Paris
...,...,...,...
6212,Mansle,France,Marseille
6213,Marne-La-Vallee,France,Marseille
6214,Tuscany,Italy,Turin
6215,Tuscany,Italy,Turin


In [19]:
app_data = df_merge[['icity','icoun','GlobalCity']].drop_duplicates()
print(app_data)
app_data.to_excel("Projects_City_name_diff.xlsx")

                 icity        icoun  GlobalCity
0     Paris La Défense       France       Paris
1          Paris 15Eme       France       Paris
2     Paris La Defense       France       Paris
5       Paris Cedex 16       France       Paris
18       Parcay Meslay       France       Paris
...                ...          ...         ...
6210        Malestroit       France   Marseille
6211            Mansle       France   Marseille
6213   Marne-La-Vallee       France   Marseille
6214           Tuscany        Italy       Turin
6216          Westgate  New Zealand  Wellington

[700 rows x 3 columns]


# Main Step


In [21]:
city_diff = pd.read_excel('Projects_City_name_diff.xlsx')

In [146]:
for index,r in city_diff.iterrows():
    indexes = projects_data.loc[projects_data.icity == r.dcity].index
    index_2 = projects_data.loc[projects_data.dcity == r.dcity].index
    projects_data.at[indexes,'icity'] = r['GlobalCity']
    projects_data.at[index_2,'dcity'] = r['GlobalCity']

In [22]:
projects_data['icity_new'] = projects_data['icity']
projects_data['dcity_new'] = projects_data['dcity']
projects_data[['icity','icity_new','icoun','dcity','dcity_new','dcoun']]

,icity,icity_new,icoun,dcity,dcity_new,dcoun
0,München,München,Germany,Edinburgh,Edinburgh,United Kingdom
1,Luxembourg,Luxembourg,Luxembourg,Brussels,Brussels,Belgium
2,Asolo,Asolo,Italy,NaN,NaN,United States of America
3,Göteborg,Göteborg,Sweden,Naperville,Naperville,United States of America
4,Mclean,Mclean,United States of America,Agra,Agra,India
...,...,...,...,...,...,...
86883,Delhi,Delhi,India,NaN,NaN,Middle East
86884,Delhi,Delhi,India,NaN,NaN,Saudi Arabia
86885,Delhi,Delhi,India,NaN,NaN,Bahrain
86886,Riyadh,Riyadh,Saudi Arabia,NaN,NaN,United States of America


# Drop the existing GC columns

In [23]:
projects_data.drop(['GC1','GC2'],axis=1,inplace=True)

In [24]:
list(projects_data.columns)

['Unnamed: 0',
 'Unnamed: 0.1',
 'pheadline',
 'pnumber',
 'ptype',
 'pstatus',
 'iexpen',
 'PBD',
 'iname',
 'iBVDid',
 'gname',
 'gBVDid',
 'pname',
 'scity',
 'scoun',
 'sregion',
 'sworld',
 'dcity',
 'dcoun',
 'dregion',
 'dworld',
 'iBVDsector_primarycode',
 'iBVDsector_primarycode_desc',
 'pBVDsector_primarycode',
 'pBVDsector_primarycode_desc',
 'pBVDsector_secondarycode',
 'pBVDsector_secondarycode_desc',
 'pNAICS_primarycode',
 'pNAICS_primarycode_desc',
 'pNAICS_secondarycode',
 'pNAICS_secondarycode_desc',
 'pUSSIC_primarycode',
 'pUSSIC_primarycode_desc',
 'pUSSIC_secondarycode',
 'pUSSIC_secondarycode_desc',
 'PBdesc',
 'pmotives',
 'pmarketserved',
 'newsdate',
 'rdate',
 'adate',
 'edate',
 'cdate',
 'acdate',
 'cancelleddate',
 'pdate',
 'laststatusdate',
 'lastupdatedate',
 'ncurr',
 'pjobs',
 'pjobstype',
 'prationale',
 'pcomments',
 'iNIcalssification_desc',
 'ipost',
 'icity',
 'icoun',
 'iworld',
 'itele',
 'iweb',
 'imail',
 'gpost',
 'gcity',
 'gcoun',
 'gworld

In [27]:
gc_data_s = gc_data[['GlobalCity','Country','GC1','GC2']]
gc_data_s.columns=['icity_new','icoun','GC1_i','GC2_i']
gc_data_s

,icity_new,icoun,GC1_i,GC2_i
0,London,UK,1,1.0
1,New York,USA,1,1.0
2,Paris,France,1,1.0
3,Tokyo,Japan,1,1.0
4,Chicago,USA,1,1.0
...,...,...,...,...
119,Tijuana,Mexico,0,4.0
120,Turin,Italy,0,4.0
121,Utrecht,Netherlands,0,4.0
122,Wellington,New Zealand,0,4.0


In [43]:
#gc_data.rename(columns={'GlobalCity':'icity','GC1':'GC1_i','GC2':'GC2_i'},inplace=True)
#gc_data.columns

Index(['#', 'icity', 'Country', 'Category', 'Sub-category', 'Description',
       'GC1_i', 'GC2_i'],
      dtype='object')

In [28]:
dmerge = pd.merge(projects_data, gc_data_s,how="left" ,on=["icity_new","icoun"]) 
list(dmerge)

['Unnamed: 0',
 'Unnamed: 0.1',
 'pheadline',
 'pnumber',
 'ptype',
 'pstatus',
 'iexpen',
 'PBD',
 'iname',
 'iBVDid',
 'gname',
 'gBVDid',
 'pname',
 'scity',
 'scoun',
 'sregion',
 'sworld',
 'dcity',
 'dcoun',
 'dregion',
 'dworld',
 'iBVDsector_primarycode',
 'iBVDsector_primarycode_desc',
 'pBVDsector_primarycode',
 'pBVDsector_primarycode_desc',
 'pBVDsector_secondarycode',
 'pBVDsector_secondarycode_desc',
 'pNAICS_primarycode',
 'pNAICS_primarycode_desc',
 'pNAICS_secondarycode',
 'pNAICS_secondarycode_desc',
 'pUSSIC_primarycode',
 'pUSSIC_primarycode_desc',
 'pUSSIC_secondarycode',
 'pUSSIC_secondarycode_desc',
 'PBdesc',
 'pmotives',
 'pmarketserved',
 'newsdate',
 'rdate',
 'adate',
 'edate',
 'cdate',
 'acdate',
 'cancelleddate',
 'pdate',
 'laststatusdate',
 'lastupdatedate',
 'ncurr',
 'pjobs',
 'pjobstype',
 'prationale',
 'pcomments',
 'iNIcalssification_desc',
 'ipost',
 'icity',
 'icoun',
 'iworld',
 'itele',
 'iweb',
 'imail',
 'gpost',
 'gcity',
 'gcoun',
 'gworld

In [29]:
gc_data_s.columns=['dcity_new','dcoun','GC1','GC2']
gc_data_s

,dcity_new,dcoun,GC1,GC2
0,London,UK,1,1.0
1,New York,USA,1,1.0
2,Paris,France,1,1.0
3,Tokyo,Japan,1,1.0
4,Chicago,USA,1,1.0
...,...,...,...,...
119,Tijuana,Mexico,0,4.0
120,Turin,Italy,0,4.0
121,Utrecht,Netherlands,0,4.0
122,Wellington,New Zealand,0,4.0


In [30]:
final_merge = pd.merge(dmerge, gc_data_s,how="left" ,on=["dcity_new","dcoun"]) 
list(final_merge)

['Unnamed: 0',
 'Unnamed: 0.1',
 'pheadline',
 'pnumber',
 'ptype',
 'pstatus',
 'iexpen',
 'PBD',
 'iname',
 'iBVDid',
 'gname',
 'gBVDid',
 'pname',
 'scity',
 'scoun',
 'sregion',
 'sworld',
 'dcity',
 'dcoun',
 'dregion',
 'dworld',
 'iBVDsector_primarycode',
 'iBVDsector_primarycode_desc',
 'pBVDsector_primarycode',
 'pBVDsector_primarycode_desc',
 'pBVDsector_secondarycode',
 'pBVDsector_secondarycode_desc',
 'pNAICS_primarycode',
 'pNAICS_primarycode_desc',
 'pNAICS_secondarycode',
 'pNAICS_secondarycode_desc',
 'pUSSIC_primarycode',
 'pUSSIC_primarycode_desc',
 'pUSSIC_secondarycode',
 'pUSSIC_secondarycode_desc',
 'PBdesc',
 'pmotives',
 'pmarketserved',
 'newsdate',
 'rdate',
 'adate',
 'edate',
 'cdate',
 'acdate',
 'cancelleddate',
 'pdate',
 'laststatusdate',
 'lastupdatedate',
 'ncurr',
 'pjobs',
 'pjobstype',
 'prationale',
 'pcomments',
 'iNIcalssification_desc',
 'ipost',
 'icity',
 'icoun',
 'iworld',
 'itele',
 'iweb',
 'imail',
 'gpost',
 'gcity',
 'gcoun',
 'gworld

In [31]:
projects_f = final_merge.iloc[:,2:]
projects_f

,pheadline,pnumber,ptype,pstatus,iexpen,PBD,iname,iBVDid,gname,gBVDid,...,bmi_female_d,bmi_both_sexes,bmi_male,bmi_female,icity_new,dcity_new,GC1_i,GC2_i,GC1,GC2
0,"BayWa to expand sales office in Edinburgh, Sco...",999379749,Expansion,Completed Assumed,1.91,Sales Office,BAYWA RE RENEWABLE ENERGY GMBH,DE8170725443,BAYERISCHE RAIFFEISEN-BETEILIGUNGS-AG,DEFEB18938,...,29.2 [25.8-32.8],24.7 [21.3-28.2],25.4 [20.6-30.6],24.1 [19.5-28.9],München,Edinburgh,NaN,NaN,NaN,NaN
1,Vilebrequin to open clothing store in Brussels...,953988958,New,Completed Assumed,0.82,Retail,TRB INTERNATIONAL SA,LULB26186,G-111 APPAREL GROUP LTD,LU*B000062734,...,23.3 [18.4-28.3],23.3 [18.7-28.1],24.8 [18.0-32.3],21.8 [15.6-28.4],Luxembourg,Brussels,0.0,3.0,1.0,2.0
2,"Replay to open clothing store in New York, USA",946529918,New,Completed Assumed,1.31,Retail,FASHION BOX SPA,IT03676290269,EQUIBOX HOLDING SPA,IT04339950265,...,36.9 [32.2-41.7],22.1 [19.0-25.5],21.5 [17.1-26.3],22.7 [18.1-27.6],Asolo,NaN,NaN,NaN,NaN,NaN
3,SKF to establish new research and development ...,941723112,New,Completed Assumed,30,R&D Centre,SKF AB,SE5560073495,FAM AB,SE5567400246,...,36.9 [32.2-41.7],21.2 [17.8-24.6],23.1 [18.2-28.2],19.4 [14.8-24.2],Göteborg,Naperville,NaN,NaN,NaN,NaN
4,"Hilton Worldwide to open hotel in Agra, India",931655359,New,Completed Assumed,21.77,Hotels,HILTON WORLDWIDE HOLDINGS INC.,US274384691,HILTON WORLDWIDE,US362058176,...,4.4 [3.2-5.9],36.0 [32.5-39.5],35.0 [30.2-40.1],36.9 [32.2-41.7],Mclean,Agra,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86883,Oyo plans to open Oyo Hotels & Homes brand hot...,562372036,New,Rumour,n.a.,Hotels,ORAVEL STAYS PVT LTD,IN0008565743,ORAVEL STAYS PVT LTD,IN0008565743,...,NaN,NaN,NaN,NaN,Delhi,NaN,NaN,NaN,NaN,NaN
86884,Oyo plans to open Oyo Hotels & Homes brand hot...,523642464,New,Rumour,n.a.,Hotels,ORAVEL STAYS PVT LTD,IN0008565743,ORAVEL STAYS PVT LTD,IN0008565743,...,NaN,NaN,NaN,NaN,Delhi,NaN,NaN,NaN,NaN,NaN
86885,Oyo plans to open Oyo Hotels & Homes brand hot...,387828716,New,Rumour,n.a.,Hotels,ORAVEL STAYS PVT LTD,IN0008565743,ORAVEL STAYS PVT LTD,IN0008565743,...,NaN,NaN,NaN,NaN,Delhi,NaN,NaN,NaN,NaN,NaN
86886,Saudi Basic plans to open a methanol manufactu...,327369715,New,Completed Assumed,119.15,Manufacturing,SAUDI BASIC INDUSTRIES CORPORATION SJSC,SA30020GS,SAUDI BASIC INDUSTRIES CORPORATION SJSC,SA30020GS,...,NaN,NaN,NaN,NaN,Riyadh,NaN,0.0,3.0,NaN,NaN


In [32]:
projects_f.to_excel("Projects_2020_Latest.xlsx")

In [110]:
# d=[]
# for index,c in gc_data.iterrows():
#    a = projects_data[(projects_data['dcoun'] == c.Country)  & ((projects_data['dcity'] != c['GlobalCity']) & (projects_data['dcity'].str[:2] == c['GlobalCity'][:2]))][['dcity','dcoun']]
#    a['GlobalCity'] = c['GlobalCity']
#    d.append(a) 
   

# appended_data = pd.concat(d)
# print(appended_data)

                 dcity        dcoun  GlobalCity
8595            Pantin       France       Paris
30630        Tomakomai        Japan       Tokyo
35673         Tokoname        Japan       Tokyo
47054        Tomakomai        Japan       Tokyo
51287        Tomakomai        Japan       Tokyo
...                ...          ...         ...
63292           Mansle       France   Marseille
68572  Marne-La-Vallee       France   Marseille
23951          Tuscany        Italy       Turin
75896          Tuscany        Italy       Turin
25106         Westgate  New Zealand  Wellington

[2050 rows x 3 columns]


In [111]:
# appended_data.to_excel("City_name_diff.xlsx")

In [123]:
# i=[]
# for index,c in gc_data.iterrows():
#    b = projects_data[(projects_data['icoun'] == c.Country)  & ((projects_data['icity'] != c['GlobalCity']) & (projects_data['icity'].str[:2] == c['GlobalCity'][:2]))][['icity','icoun']]
#    b['GlobalCity'] = c['GlobalCity']
#    i.append(a) 
   

# append_data = pd.concat(i)
# print(append_data)
# appended_data.to_excel("projects_City_name_diff_icity.xlsx")

          dcity        dcoun  GlobalCity
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
...         ...          ...         ...
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington
25106  Westgate  New Zealand  Wellington

[123 rows x 3 columns]


In [33]:
projects_f.shape

(86888, 300)